In [1]:
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import os
fontpath = os.path.expanduser('~/.local/share/fonts/LinLibertine_DRah.ttf')
prop = font_manager.FontProperties(fname=fontpath)

params = {
    "axes.titlesize" : 14,
    'axes.labelsize': 12,
    'font.size': 12,
    'legend.fontsize': 12,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    #'font.family': prop.get_name(),
    'text.usetex': True
}

mpl.rcParams.update(params)

import sys
import os.path as op
# Local Modules

PATH = r'C:\Users\gbrag\OneDrive\Desktop\ADA\ada-2024-project-sickada\data\raw'
path_metadata = op.join(PATH, "yt_metadata_en.jsonl.gz")
DERIV = r'C:\Users\gbrag\OneDrive\Desktop\ADA\ada-2024-project-sickada\data'
sys.path.insert(0, os.path.abspath(PATH))
#from helpers.plot import set_size

import warnings
warnings.filterwarnings('ignore')

## Channel data 

In [ ]:
#df_sb_f = pd.read_csv("/df_timeseries_en.tsv.gz", compression="infer", sep="\t")
#df_vd_f = pd.read_feather("/dlabdata1/youtube_large/yt_metadata_helper.feather") # feather does not have title,tags, description
#df_sb_f["datetime"] = pd.to_datetime(df_sb_f["datetime"])

df_channels = pd.read_csv(op.join(PATH, "df_channels_en.tsv.gz"), compression="infer", sep="\t")
df_channels["join_date"] = pd.to_datetime(df_channels["join_date"])

In [ ]:
df_channels.category_cc.unique() # choose 'Education'

array(['Gaming', 'Education', 'Entertainment', 'Howto & Style', 'Sports',
       'Music', 'Film and Animation', 'Comedy', 'Nonprofits & Activism',
       'People & Blogs', 'News & Politics', 'Science & Technology',
       'Pets & Animals', 'Autos & Vehicles', 'Travel & Events', nan],
      dtype=object)

In [5]:
df_edu = df_channels.loc[df_channels['category_cc'].isin(['Education'])]
df_edu

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.0870
22,Education,2013-02-09,UCBnZ16ahKA2DZ_T5W0FPUXg,ChuChu TV Nursery Rh...,26600000,317,44.0,2.0870
27,Education,2011-12-14,UCcdwLMPsaU2ezNSJU1nFoBQ,Pinkfong! Kids' Song...,23200000,1382,55.0,2.0870
39,Education,2013-10-25,UCRx3mKNUdl8QE06nEug7p6Q,Billion Surprise Toy...,24700000,150,83.0,2.0870
46,Education,2011-06-22,UCKAqou7V9FAWXpZd9xtOg3Q,Little Baby Bum - Nu...,19900000,958,93.0,2.0870
...,...,...,...,...,...,...,...,...
136282,Education,2012-11-21,UCccoliC479T88ziS1TxeLeg,skunkpaste,10074,146,977576.0,53.1435
136291,Education,2013-09-05,UCu13MNNOLhyRV-YtlYWV4tw,Matthew Williams,10000,374,977915.0,53.1435
136316,Education,2015-08-12,UCRoZf8DdJzF5Xi99gtFDY0A,Geoid,10276,136,979269.0,53.1435
136332,Education,2013-09-10,UCax581uo5yNzdgODBdH67cw,Bond Robin,10225,469,980820.0,53.1435


In [ ]:
# faire des sous catégories dans education, 
# plot : Camembert avec les differentes sous categories, rayon est le nombre total 
# how to - match channels with their videos and try to get the theme of them by using the description, title and tags fiels

## Extraction youtube metadata

In [2]:
import random
import json
import gzip

def filter_jsonl(input, category, batch_size, random_seed, all = False):
    filtered_data = []
    random.seed(random_seed)
    counter = 0
    with gzip.open(input, 'rt', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)

            if entry.get('categories') == category:
                counter +=1
                if len(filtered_data) <= batch_size or all:
                    filtered_data.append(entry)
                else:
                    index_to_replace = random.randint(0, len(filtered_data) - 1)
                    if index_to_replace < batch_size:
                        filtered_data[index_to_replace] = entry
      
                
    print(f"There are {counter} videos in the Education category!")
    return pd.DataFrame(filtered_data)

            

df = filter_jsonl(path_metadata, 'Education', 500000, 0, all = False)

df.to_csv(op.join(DERIV, 'df_edu_all.csv'))

There are 3795564 videos in the Education category!


## Preprocessing

In [3]:
import re
df = pd.read_csv(op.join(DERIV, 'df_edu_all.csv'), index_col=0)
df['desc_clean'] = df['description'].fillna('')
df['text'] = df['title'].fillna('') + " " + df['tags'].fillna('') 

#sort the description separately since way more garbage, some titles contain facebook and twitter and others that we are interested to remove

In [3]:
pd.set_option('display.max_colwidth', 80)
df.sample(5)[['desc_clean', 'text']]
# 32379 crypto video with lots of links
# 35962 marseillaise
# 22678 41830 non alpha and indian
# 34547 numbers accolated
# 17337 very long strings in crypto vids
# 3235 hindu

,desc_clean,text
34620,"A video, that describes, who, I believe, Jesus, is, to me.","Who Jesus Is To Me (Automated Voice) God’sTruthBeingTold,who Jesus is to m..."
35217,"Tim Crane gives a general overview of analytic philosophy and its nature, fo...","Analytic Philosophy: Its History, Science, & Logic Philosophy,Analytic Philo..."
35816,"Get Free Korean Lessons on your Android, iPhone, iPad or Kindle Fire! Click ...","Learn the Top 25 Korean Adjectives Korean Language (Human Language),learn Ko..."
35291,Please thank Lucky Gunner for bringing us today’s video of VangComp Makes Ex...,"VangComp Makes Excellent Shotguns asp,cover your asp,active self protection,..."
10905,Reviewing the EQ 73 and EQ 81 plugins from IK Multimedia.,IK Multimedia EQ 73 & EQ 81 Review - Audio Plugin IK Multimedia (Business Op...


In [4]:
df

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,desc_clean,text
0,Education,UCs7on9W7SIbyO4f-Pb7lgbg,2019-11-10 17:01:54.661515,Quantity is something that you can measure using any container that you have...,6.0,oHtYkElKf_o,179,47.0,"learn maths - jugs and mugs,volume,size,maths,mathematics,cbse,icse,ssc,stat...",Learn Maths - Jugs and Mugs,2012-03-28 00:00:00,25380.0,Quantity is something that you can measure using any container that you have...,"Learn Maths - Jugs and Mugs learn maths - jugs and mugs,volume,size,maths,ma..."
1,Education,UCsJbboIjkvAkxbnEu6w3nyg,2019-11-13 20:48:26.093168,NaN,0.0,MQ9MGdm7Brs,240,4.0,"African,Odyssey,marionette puppet,puppets,Dance,Culture,Entertainment,The Ke...",African Odyssey: Part 1,2012-06-29 00:00:00,511.0,,"African Odyssey: Part 1 African,Odyssey,marionette puppet,puppets,Dance,Cult..."
2,Education,UCsgkhimI0MthrMZo-F2zrTQ,2019-11-19 20:33:30.552035,When Gov becomes an exclusive club where people in Gov only care about thems...,3.0,iWWqX-K-17U,1695,165.0,"rick gore,firearm shooting,gun tactics,deadly force,survivalist training,dea...",Presidential Impeachment History - Has It Been Used Too Much or Not Enough?,2016-09-15 00:00:00,2456.0,When Gov becomes an exclusive club where people in Gov only care about thems...,Presidential Impeachment History - Has It Been Used Too Much or Not Enough? ...
3,Education,UC-9wdVL1KR6xcTl5hw0nDeg,2019-11-20 01:14:01.147695,Justin Zweifel was looking for a competitive edge in the fisheries and wildl...,0.0,CAtS9CKd85A,32,2.0,"Oregon State University,Oregon State Ecampus,Natural resources,Fisheries and...","Justin Zweifel, Master of Natural Resources | Oregon State Ecampus",2016-05-18 00:00:00,451.0,Justin Zweifel was looking for a competitive edge in the fisheries and wildl...,"Justin Zweifel, Master of Natural Resources | Oregon State Ecampus Oregon St..."
4,Education,UCsCDb4ra5i5ZCfC4SprEEAQ,2019-11-08 22:10:22.347031,★★★Sign up to the Weekly Book Summary Newsletter: https://mailchi.mp/632a940...,0.0,B0UOj24mLi4,772,6.0,"50 words to your dreams,michael george knight,50 words to your dreams chapte...",50 Words to Your Dreams | Chapter 11: Mind | Michael George Knight | bestboo...,2018-11-02 00:00:00,210.0,★★★Sign up to the Weekly Book Summary Newsletter: https://mailchi.mp/632a940...,50 Words to Your Dreams | Chapter 11: Mind | Michael George Knight | bestboo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49996,Education,UC-V6odR7HzLCuqjYeowPjLA,2019-11-20 09:04:45.061296,"Jeffrey C. Hall, 2017 Nobel Prize Laureate praises the role of the humble fr...",2.0,AWbxsTm9fgQ,309,81.0,NaN,"“The key fourth awardee here is, as some of us call them, the little fly.”",2017-10-02 00:00:00,7003.0,"Jeffrey C. Hall, 2017 Nobel Prize Laureate praises the role of the humble fr...","“The key fourth awardee here is, as some of us call them, the little fly.”"
49997,Education,UCsCj4BzRpOzWMWxrNBJyeVw,2019-11-12 23:45:26.658442,Ken and I celebrated New Years Eve with our boys in Cabo San Lucas. The even...,NaN,CPfUE3lH_24,181,NaN,"new years,new years eve,new years eve celebration,2018,ring in 2018,the ball...",Happy New Year 2018 | To Making It Count,2018-01-01 00:00:00,1422.0,Ken and I celebrated New Years Eve with our boys in Cabo San Lucas. The even...,"Happy New Year 2018 | To Making It Count new years,new years eve,new years e..."
49998,Education,UCsCyJ7xl8rQ10Cyz-fwJ5Xg,2019-11-02 19:28:56.151117,By: Alistair Begg www.truthforlife.org,0.0,2xe-v4ch59c,245,2.0,YouTube editor,Case Closed Excerpt,2011-08-17 00:00:00,176.0,By: Alistair Begg www.truthforlife.org,Case Closed Excerpt YouTube editor
49999,Education,UC-WHWhLDdRAiiT53kqDe3PA,2019-11-19 21:00:15.263208,"Nephron is the structural and functional unit of kidney,\n\nTo help students...",7.0,R33vdiYbbkg,691,404.0,"Nephron,PCT,DCT,loop of Henle,Bowman's capsule,glomerulus,Malpighian body,af...",Excretion and Osmoregulation - Huma

##### debug

In [4]:
df[df['title'].str.contains('twitter')]['title']

33993     Twitter Ads 2019 | How to run Followers ad Cam...
36741     Pkrussl is NOT transphobic - twitter is just dumb
47320      understanding twitter marketing - quick overview
153573    CA Inter Result Special | Dil Ki Baat |  #askc...
174123    How to use twitter in jio phone, Jio feature p...
218918    digital marketing tip1 oct3 twitonamy twitter ...
224199    How to use twitter analytics | Twitter Analyti...
227636      re:publica 2018 – Bild getwittert: Tatverdacht!
247857             14 (Twitter)  How to tweet using twitter
257367    Hashtagify | Finding Hashtags | How to find to...
346688    ❤️❤️Personal Talk | Not related to CA | Questi...
358551    CA Inter Result Special | Dil Ki Baat |  #askc...
359052                               twitter conversation 1
403921    how to enable dark mode on twitter ? Twitter d...
410091            How to change url of your twitter account
465499    Before Result Live Session | #askcaneeraj on t...
Name: title, dtype: object

In [ ]:
string = df['description'].iloc[5046].lower()
print(string)
remove_urls(string)#.replace('website', 'website blab bla truc muche promotion')

#### code

In [6]:
# everything to lower case so that same words are treated the same
df['text'] = df['text'].apply(lambda x: x.lower() if isinstance(x,str) else x)
df['desc_clean'] = df['desc_clean'].apply(lambda x: x.lower() if isinstance(x,str) else x)

In [7]:
# remove urls links and their associated text
url_pattern = re.compile(r'(https?://\S+|www\.\S+)') 
com_pattern = re.compile(r'([^\s]+\.com)')
fb_pattern = re.compile(r'(facebook\s+page|facebook\s+group)[^\w\s]*.*?(\n|$)')
#lines that start with brands or websites 
link_pattern = re.compile(r'(\n+)(facebook|twitter|pinterest|tumblr|instagram|website|amazon)[^\w\s]*\s+\S+')
link_pattern2 = re.compile(r'(\n+)(facebook|twitter|pinterest|tumblr|instagram|website|amazon)')

#TODO lines that contain these websites in the middle of text - hard to implement since not sure if discard

long_words = re.compile(r'\b[a-zA-Z0-9]{21,}\b') # most words in english are below 20 letters, bigger than that is a crypto wallet id

def remove_urls(text, desc = False):
        text = url_pattern.sub('', text)
        text = com_pattern.sub('', text)
        text = long_words.sub('', text)
        if desc: # titles and tags should keep brand names since these might be the focus of the video
                text = fb_pattern.sub('', text)
                text = link_pattern.sub('', text)
                return link_pattern2.sub('', text)
        return text

df['desc_clean'] = df['desc_clean'].apply(remove_urls, desc = True)
df['text'] = df['text'].apply(remove_urls)
#test = 'https://www.youtube.com/everydaytacticalvids\n my twitter account - https://twitter.com/everydaytactic1\n my facebook group tha tha \n ow'
#remove_urls(test, True)

In [8]:
def clean_non_word(text): #punctuation, underscores seem to evade this regex so add it
        return  re.sub(r'[^\w\s]|_+', ' ', text)

def clean_non_ascii(text): # indian symbols that might still be left
        return  re.sub(r'[^\x00-\x7F]+', ' ', text)

df['desc_clean'] = df['desc_clean'].apply(clean_non_word).apply(clean_non_ascii)
df['text'] = df['text'].apply(clean_non_word).apply(clean_non_ascii)

In [9]:
def clean_numeric(text):
    return re.sub(r'\d+(?![a-zA-Z])', '', text) # numbers that are not accolated to strings : # TODO MAybe all numbers
df['desc_clean'] = df['desc_clean'].apply(clean_numeric)
df['text'] = df['text'].apply(clean_numeric)

In [10]:
def clean_space_newline(text):
    return re.sub(r'\s{2,}', ' ', text.replace('\n', ' ')).strip()

df['desc_clean'] = df['desc_clean'].apply(clean_space_newline)
df['text'] = df['text'].apply(clean_space_newline)

In [11]:
df['text_clean'] = df['text'] + " "+ df['desc_clean']
df = df.drop(['text', 'desc_clean'], axis = 1)
df.to_csv(op.join(DERIV, 'df_edu_500k_clean.csv'))

In [12]:
#df= pd.read_csv(op.join(DERIV, 'df_edu_all_clean.csv'))
#optional : make it lighter 
df = df.drop(['description'], axis = 1)

In [ ]:
# no need to perform lemmatization, stemming or stopword removal since BERT handles it 

# Categorize subtopics

In [14]:
trial = df#.copy().sample(5000)
pd.set_option('display.max_colwidth', 80)
trial = trial.reset_index()

In [15]:
trial.head(5)

,index,categories,channel_id,crawl_date,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count,text_clean
0,0,Education,UCElzlyMtkoXaO3kFa5HL0Xw,2019-11-18 23:15:07.993741,1.0,446aatZJ-ak,386,2.0,"gitlab tutorial,gitlab tutorial for beginners windows,gitlab create issue,cr...",Creating A Gitlab Issue,2018-04-24 00:00:00,494.0,creating a gitlab issue gitlab tutorial gitlab tutorial for beginners window...
1,1,Education,UCDqaMkyqQ8zbFC5kvYrM8CQ,2019-11-14 17:20:44.688113,11.0,Sy6FPoBvaYg,228,471.0,"venus flytrap,how to water Orchids,growing Orchids,how,to,care,for,orchids,h...",NEW PLANTS IN THE GREENHOUSE! MASSIVE AIR PLANT AND BIG HOYAS,2015-02-07 00:00:00,26656.0,new plants in the greenhouse massive air plant and big hoyas venus flytrap h...
2,2,Education,UC3xE9rEV485FTlt937mv1Ig,2019-10-30 01:15:40.021510,34.0,RGccaNqE6hQ,509,346.0,NaN,Math Tricks & Shortcuts for All Upcoming Exams - Part 2 - Explain in Bengali...,2018-03-26 00:00:00,8456.0,math tricks shortcuts for all upcoming exams part explain in bengali by deba...
3,3,Education,UCA7OQkX9AEIVQ6j9i0OSQhA,2019-11-23 11:17:12.567113,1.0,c-wTLMLtJYQ,3590,8.0,"Beginnings of National Movement : Society and Culture,CEC,CEC Edusat,CEC New...",Beginnings of National Movement : Society and Culture,2016-11-17 00:00:00,435.0,beginnings of national movement society and culture beginnings of national m...
4,4,Education,UCt_t6FwNsqr3WWoL6dFqG9w,2019-11-18 23:57:27.805714,18.0,al4L9bcWsNM,238,409.0,"tdcs,brain stimulation,Neuroscience (Field Of Study),Neurology (Medical Spec...",DIY Brain Enhancement or Frankenstein?,2014-09-02 00:00:00,18249.0,diy brain enhancement or frankenstein tdcs brain stimulation neuroscience fi...


In [ ]:
# Try to extract sub topics with sentence BERt like transformer s
from sentence_transformers import SentenceTransformer # https://sbert.net/
from sklearn.cluster import KMeans


print('Encoding model...')
model = SentenceTransformer('all-MiniLM-L6-v2') 
embeddings = model.encode(trial.get('text_clean'))

# Step 2: Perform K-Means clustering
print('Starting kmeans...')
num_clusters = 50
kmeans = KMeans(n_clusters=num_clusters, random_state=0)

print('Fitting model...')
clusters = kmeans.fit(embeddings)
cluster_assignment = clusters.labels_

print('Predicting labels...')
trial['subcategories'] = clusters.predict(embeddings)

Encoding model...


In [670]:
for cluster_num in range(num_clusters):
    print(f"Cluster {cluster_num}")
    cluster_data = trial[trial['subcategories'] == cluster_num]
    display(cluster_data.sample(5)[['title', 'description', 'tags']])

Cluster 0


,title,description,tags
397,Simple probability problem (genetics),NaN,"genetics,law of probability,statistics"
1116,Evolve: The Game of Unnatural Selection (Kickstarter),"Find out more here on Kickstarter: http://dld.bz/dk6gp\n\nBuild over 8,000 c...","Evolution,Game,Educational,Card Game,Kickstarter (Website),Evolve,Classroom,..."
831,How to solve phenotype-genotype problems,NaN,"Genetics,DNA,Phenotype,Genotype"
1633,ABO blood group problem,NaN,"Molecular Biology,Alleles,Genomics,Genetics,Eukaryotic,Genetic Code,Homozygo..."
1381,In which direction new strand of the DNA grows?,NaN,"Genetics,DNA,cytosine,guanine,thymine,adenine,uracil"


Cluster 1


,title,description,tags
510,Ceylon @ the kent freedom movement Health,"ceylon talks about many cures for almost everything\nPublished on Feb 23, 20...","David Icke,Chunky Mark,Spiritualentertainer,TheAntiTerrorist,Rob Menard,Dean..."
1960,"Dr Daniel Condron : Atlantis, Esoteric Wisdom & Spiritual Insights 2018","View in HD~Earth-Keeper 2018 Presents: Author, Academic, Teacher & Master Me...","Daniel Condron,Earth-Keeper,Ancient Alietns,Atlantis,Arklantis,Coast to Coas..."
393,Hilary Putnam on Meaning & Externalism,Hilary Putnam was awarded The Rolf Schock Prize in Logic and Philosophy by t...,"Philosophy,Hilary Putnam,Analytic Philosophy,History of Philosophy,Epistemol..."
250,The Life & Work of Frank P. Ramsey,"Frank P. Ramsey was a British philosopher, mathematician, economist, who liv...","Philosophy,Analytic Philosophy,History of Philosophy,Mathematical Logic,Foun..."
991,Heidegger on Marx and World Change (1969),A clip of Martin Heidegger from a 1969 interview with Richard Wisser discuss...,"Philosophy,Heidegger,Marx,History of Philosophy,Interview,Clip,German Philos..."


Cluster 2


...

,title,description,tags
1158,Android Tutorial for Beginners 30 # Action Bar (ActionBar) # Overflow Menu ...,Searches related to actionbar overflow\nactionbar overflow menu style\nactio...,"Android (Operating System),Java (Programming Language),Java Development Kit ..."
1514,Project PC Build - 16 Things I Learned & Tips And Advice - Part 4,Tips and advice on building a PC in 2018 from someone who hasn't built a PC ...,"building pc,tips and advice,new pc build for beginners,thermal paste,intel 7..."
658,Armory Engine 0.5 Endless Runner Tutorial,Here is a new Blender Armory 3D Engine 0.5 tutorial in which I explain the s...,"Armory 0.5,Armory 3d Engine,Armory 3d tutorial,Armory Endless Runner,Endless..."
950,Unity game making (3) : Graphics & Level Design,Here is part 3 of making a Game with Unity series. In this one I will show y...,"make a game with unity,unity game graphics,unity assets free,unity 3d level ..."
474,ZBrush & Blender : GoZ Applink GoB,In this video I show the workflow to go from Zbrush to Blender and vice vers...,"Blender ZBrush GoZ,Blender Zbrush GoB,GoB Applink Blender,Blender addon GoB,..."


Cluster 38


,title,description,tags
1310,Chess Strategy: Importance of centre,♕ Quick Success in Chess: http://chess-teacher.com/free-training-course/\n♕ ...,"chess,centre,chess strategy,centre control,chess centre,pawn,chess pawns,con..."
225,How Robert Herjavec Went From Broke Immigrant to $200 Million Dollar Mogul,Stock Market Mastery Course: http://bit.ly/2hurfQO\n\nWealth Accelerator Cou...,"Jack,Chapple,robert herjavec,robert herjavec net worth,herjavec group,entrep..."
111,Zig Ziglar: Secrets of Closing the Sale Book Summary,★★★Sign up to the Weekly Book Summary Newsletter: https://mailchi.mp/632a940...,"Secrets of Closing the Sale by Zig Ziglar,secrets of closing the sale zig zi..."
1639,What You Were Taught About Starting a Business is WRONG!,Stock Market Mastery Course: http://bit.ly/2hurfQO\n\nWealth Accelerator Cou...,"Jack,Chapple,starting a business,startup cat,how to start a business,how to ..."
310,Forex Trading: Identifying Stoploss Runs In the Forex Market,http://extraordinarytrading.com/membership-account/membership-levels/\n\nTra...,"forex trading,forex trader,how to trade forex,trading psychology,forex techn..."


Cluster 39


,title,description,tags
1239,Informatica MDM Introduction,http://www.bisptrainings.com/course/Informatica-MDM\n\n\n\nBISP is most trus...,"MDM training,Informatica MDM Training Online,Informatica MDM Course Details,..."
482,Preparing for exams?? This will help you in saving your time.,Do let me know in the comment box what is your view on it!,"icai,institute of chartered accountants of india,ca ipcc,ca neeraj arora,ca ..."
1370,How to Study EIS | MOTIVATION FOR MAY 2019,EIS course is available at discount now. Register for the course before the ...,"eis made easy by pooja sharma,eis by pooja sharma,eis memory techniques,memo..."
226,Iken All-in-One Teaching System by Mexus Education - Features,Mexus Education proud to introduce Iken All-In-One Teaching System. Iken Sch...,"Smart Solution for Schools,Teachers. Parents"
1684,"How to download 2018-2019 new textbook for 11th,9th in mobile",இந்த காணொளி சமச்சீர் புதிய பாடத்திட்ட நூல்களை மின்நூலாக எவ்வாறு download செய...,"11th,textbook download,SAMACHEER book,new book,tnbook,new syllabus,9th,6th,t..."
